# Python入門第4回<div style="text-align: right; font-size: 50%;">2021年6月29日火曜日18:00-19:30</div>

台湾の院内心肺停止患者796名の臨床的情報と退院時の神経学的予後のデータ(https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0202938 )から前回作成した`cpa_data_preprocessed.csv`を使って、実際に臨床情報から患者の退院時の神経学的予後を予測する機械学習モデルを作成したいと思います。

## 1.  Numpyの紹介

実際にモデル作成の工程に移る前に、numpyモジュールのndarrayというデータ形式について説明します。numpyは数値計算を効率的に行うためのモジュールであり、行列計算をはじめとした計算操作に非常に便利なモジュールです。そこで主に扱われるndarray形式のデータは、今回直接計算には使いませんが、Pythonを用いた機械学習領域では頻用されること、今回の機械学習モジュールでも返り値がndarray形式で返されることがあることから、簡単に紹介しておきます。  

まず、下記のセルでnumpyライブラリをnpという名前でインポートします。

In [1]:
import numpy as np

練習用にtest_listというリストデータ型のオブジェクトを作ります。

In [ ]:
test_list = [0, 1, 2, 3, 4, 5]

In [ ]:
print(test_list)

In [ ]:
type(test_list)

numpyのarray関数を使って、test_listをndarrayというデータ型のオブジェクトに作り替えてみます。

In [ ]:
test_list_np = np.array(test_list)

In [ ]:
print(test_list_np)

In [ ]:
type(test_list_np)

リストデータ型のオブジェクトとndarrayオブジェクトについて計算上の違いを確認してみます。

まずリストデータ型で簡単な演算子を使ってみます。

In [ ]:
test_list + test_list

In [ ]:
test_list * 2

リスト同士は掛け算できません。下記を実行するとエラーが出ます。

In [ ]:
test_list * test_list

次にndarrayで簡単な演算子を使ってみます。ndarrayでは要素同士の計算をしてくれます。

In [ ]:
test_list_np + test_list_np

In [ ]:
test_list_np * 2

In [ ]:
test_list_np * test_list_np

ndarrayはreshapeメソッドで変形することができます。

In [ ]:
test_list_np_reshape = test_list_np.reshape(2,3)
test_list_np_reshape

ndarrayは`.shape`メソッドで次元数を確認できます。

In [ ]:
test_list_np_reshape.shape

ndarrayはどんな次元数であっても、計算するndarray同士の次元数があっていれば要素同士の計算をしてくれます。

In [ ]:
test_list_np_reshape * test_list_np_reshape

ndarrayは`.T`で転置できます。

In [ ]:
test_list_np_reshape.T

ndarrayは、計算するndarray同士の次元数が異なっていても、いずれかの次元が合致していれば他の足りない部分まで自動で補完して要素同士の計算をしてくれます。下は3行2列のデータと1行2列のデータの掛け算です。

In [ ]:
test_list_np_reshape.T * np.array([1, 10])

**練習問題1**：numpyのarray関数を使って、配列`[0, 1, 2]`と`[3, 4, 5]`をそれぞれ等しい位置にある整数どうしを掛け合わせて並べた配列を作成してください。

## 2. 機械学習(教師あり学習)による予測モデル作成

機械学習とは、データからパターンを機械的に抽出させ、それによってアルゴリズムを作成させる手法です。その中でも教師あり学習と呼ばれる方法は、既存の答えの分かっているデータから予測モデルを作成する際に利用されます。

### 2-1. 機械学習のためのデータ分割

ここからは前回前処理を行って作成した`cpa_data_preprocessed.csv`を使って、機械学習を行って予測モデルを作っていきます。データフレームを使うため、まずpandasをimportし、pd.read_csv関数を使って`cpa_data_preprocessed.csv`をデータフレーム形式で読み込みます。

In [ ]:
import pandas as pd  

In [ ]:
df = pd.read_csv('cpa_data_preprocessed.csv')

読み込んだdfを表示してみます。

In [ ]:
df

`df.info()`で概観を再度確認します。

In [ ]:
df.info()

#### 2-1-1. ラベルデータの分離

予測したいターゲットはPrognosisのカラムの情報です。Prognosisのカラム以外の情報を使ってPrognosisを予測するというモデルを作っていきます。そのために、まずデータフレームを「Prognosis以外のカラム」と「Prognosisのカラム」に分ける必要があります。

Prognosis以外の全てのカラムを持ったデータフレームは下記のように作成できます。

In [ ]:
df.drop('Prognosis', axis=1)

**練習問題2**：Prognosis以外の全てのカラムを持ったデータフレーム`df.drop('Prognosis', axis=1)`を`X`という変数に代入してください。

`X.info()`でPrognosisのカラムがなくなっているか、確認しておきます。

In [ ]:
X.info()

次に「Prognosisのカラム」だけを抜き出します。

In [ ]:
df['Prognosis']

**練習問題3**：Prognosisのカラム`df['Prognosis']`を`y`という変数に代入してください。

作成された`y`の中身を簡単に確認しておきます。

In [ ]:
y

#### 2-1-2. モデル作成用データとチューニングデータの作成

機械学習(教師あり学習)では、データを基本的に3つ準備する必要があります。学習用データ、チューニング用データ、テスト用データです。テスト用データは外部のデータの方が望ましいですが、手元のデータの一部を切り分けて作ることもあります。  
モデルの作成の流れは、学習用データを使ってモデルを作成し、チューニング用データでその性能を評価し、モデルの調整をして再度チューニング用データで性能を評価して・・・という流れになります。

<img src="data_split.png" alt="data_split" style="width: 80%;"/>

一般的にモデル作成用データの中でチューニング用データと学習用データ(チューニング用データ以外の残りのデータ)の組み合わせを入れ替えてその平均の性能でパラメータやモデルの条件を調整していきます。これをクロスバリデーションと呼びますが、今回は学習用データとチューニングデータの1つの組み合わせを作って、試しにモデルを作って評価してみるということにします。

Anacondaでは、機械学習に必要な各種モジュールを持ったscikit-learnというライブラリが既にインストールされています。ここから必要な関数を事前にimportしておきます。

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, plot_confusion_matrix, plot_roc_curve

scikit-learnの`train_test_split`関数はデータを学習用とチューニング用に分割するための関数です。これを使ってデータを学習用とチューニング用とテスト用に分割します。

scikit-learnの`train_test_split`関数はデータをモデル作成用とテスト用に分割する際、test_sizeのオプションで、テスト用データの割合を指定することができます。ここではモデル作成用に元のデータの80%、テスト用に20%を使うことにします。random_stateで分割時のランダムシートを固定化しておきます。まず、下記を実行してデータを分割します。

In [ ]:
X_model, X_test, y_model, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

さらに今作ったモデル作成用の`X_train`と`y_train`を学習用データとチューミング用データに分けます。モデル作成用のデータの80%を学習用に、20%をチューニング用に使うことにします。

同じくscikit-learnの`train_test_split`関数を使って、モデル作成用のデータ(`X_train`と`y_train`)の80%を学習用、20%をチューニング用として分割します。random_state=0で分割時のランダムシートを固定化しておきます。

In [ ]:
X_train, X_tuning, y_train, y_tuning = train_test_split(X_model, y_model, test_size=0.2, random_state=0)

できたデータの内訳は以下のようになります。

`X_train`: 学習用症例の予測に使うデータ  
`y_train`: 学習用症例の予測ターゲット(Prognosis)のデータ  
`X_tuning`: モデルのチューニング時に予測に使うデータ  
`y_tuning`: モデルのチューニング時の予測ターゲット(Prognosis)のデータ  
`X_test`: テスト時に予測に使うデータ  
`y_tuning`: テスト時の予測ターゲット(Prognosis)のデータ

`.shape`を用いて、下のデータと今分割してできたデータの行数とカラム数を確認することができます。

In [ ]:
print(X_model.shape)
print(y_model.shape)
print(X_train.shape)
print(y_train.shape)
print(X_tuning.shape)
print(y_tuning.shape)

**練習問題4**：`X_test`、`y_test`についても`.shape`を使ってデータの行数とカラム数を確認してください。

最終的にデータ分割後、データは下図のようになっていることになります。

<img src="data_split_2.png" alt="data_split_2" style="width: 80%;"/>

## 3. 機械学習モデルの作成

ここまででデータの事前準備が終わったので、実際に学習データを使って予測モデルを作り、テストデータに対する性能を評価してみます。

logistic regressionで予測モデルを作成してみます。scikit-learnからインポートした`LogisticRegression()`関数で分類器を定義して`lr`という変数に代入しておきます。LogisticRegression関数は様々なオプションが指定できますが(参考: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html )、ここではオプションに`solver='liblinear'`を設定することにします。

In [ ]:
lr = LogisticRegression(solver='liblinear')

設定した分類器を使って実際に学習データを使って学習をさせます。学習の際には`fit`メソッドを使います。`fit`メソッドは、
```
作成した分類器.fit(学習用症例の予測に使うデータ, 学習用症例の予測ターゲットのデータ)
```
のように使います。

In [ ]:
lr.fit(X_train, y_train)

学習を終えた分類器を使って、テストデータで予測をしてみます。予測の際には`predict`メソッドを使います。`predict`メソッドは、
```
学習を終えた分類器.predict(予測に使うデータ)
```
のように書いて使います。

In [ ]:
lr.predict(X_tuning)

**練習問題5**：上で作成したテストデータに対する予測結果`lr.predict(X_tuning)`を、のちに利用するために`y_pred_lr`という変数に代入してください。

事前にインポートしたscikit-learnの`accuracy_score`関数、`precision_score`関数、`recall_score`関数、`f1_score`関数を使って、それぞれ正解率、適合率、再現率、F値を確認してみます。これらのscore計算の関数は、
```
スコア計算の関数(予測ターゲットのデータ, 予測結果)
```
のように書くよう設計されています。下のセルを実行してaccuracy、precision、recall、F1 scoreを算出してください。

In [ ]:
print(accuracy_score(y_tuning, y_pred_lr))    #accuracy: 正解率
print(precision_score(y_tuning, y_pred_lr))   #precision: 適合率（陽性的中率)
print(recall_score(y_tuning, y_pred_lr))      #recall: 再現率（感度）
print(f1_score(y_tuning, y_pred_lr))          #F1 score: 再現率と適合率の調和平均 2*Precision*Recall/(Precision + Recall)

事前にインポートしたscikit-learnの`plot_confusion_matrix`関数を使って混同行列を描画してみます。`plot_confusion_matrix`関数は
```
plot_confusion_matrix(学習後の分類器、予測に使うデータ、予測ターゲットのデータ)
```
のように書いて使います。ここではオプションでさらに`values_format = 'd'`を指定して、数字が指数表示されないように設定します。ここでは具体的には以下のようになります。
```
plot_confusion_matrix(lr, X_tuning, y_tuning, values_format = 'd')
```
上記を下のセルに入れて実行し、混同行列をプロットしてみます。

In [ ]:
plot_confusion_matrix(lr, X_tuning, y_tuning, values_format = 'd')

事前にインポートしたscikit-learnの`plot_roc_curve`関数を使ってROC曲線を描画してみます。`plot_roc_curve`関数は`plot_confusion_matrix`関数と同じように
```
plot_roc_curve(学習後の分類器、予測に使うデータ、予測ターゲットのデータ)
```
のように書いて使います。

In [ ]:
plot_roc_curve(lr, X_tuning, y_tuning)

## 4. チューニングデータを用いたモデルの改善

ここまでの結果を見てみると、どうも元々のデータが不均衡(Prognosis=1が少なく、Prognosis=0が多い)であるために、分類器が全てPrognosis=0と予測する方向に偏って学習してしまっているように見えます。

不均衡データの対処はweightingやサンプリングなどの方法があります。まずここではモデルの改善のためにweightingという手法を試してみることにします。

**練習問題6**：学習器を定義するときにクラスごとのサンプル数の重みを自動で付けてくれる`class_weight='balanced'`の設定を追加してみます。この設定を追加することで、数が少ないクラス(Prognosis=1)を正解することに重視して学習が進むようになります。具体的には分類器は`LogisticRegression(solver='liblinear', class_weight='balanced')`となります。この分類器を`lr_2`という名前の変数に代入して保存してください。

**練習問題7**：作成した分類器`lr_2`に`.fit`メソッドを使って、`X_train`、`y_train`を利用してこれまでと同じように学習をさせてください。

学習が終わった分類器`lr_2`に`.predict`メソッドを使ってチューニング用データ`X_tuning`に対して予測をしてみます。

In [ ]:
lr_2.predict(X_tuning)

**練習問題8**：上で作成したテストデータに対する予測結果`lr_2.predict(X_tuning)`を、のちに利用するために`y_pred_lr_2`という変数に代入してください。

それぞれのスコア計算関数を使って、accuracy、precision、recall、F1 scoreを算出してください。下のセルを実行してください。

In [ ]:
print(accuracy_score(y_tuning, y_pred_lr_2))    #accuracy: 正解率
print(precision_score(y_tuning, y_pred_lr_2))   #precision: 適合率（陽性的中率)
print(recall_score(y_tuning, y_pred_lr_2))      #recall: 再現率（感度）
print(f1_score(y_tuning, y_pred_lr_2))          #F1 score: 再現率と適合率の調和平均　2*Precision*Recall/(Precision + Recall)

**練習問題9**：`plot_confusion_matrix`関数を使って混同行列を出力してください。`plot_confusion_matrix関数`は以下のように使うのでした。
```
plot_confusion_matrix(学習後の分類器、予測に使うデータ、予測ターゲットのデータ, values_format = 'd')
```
今回の結果を可視化するために、学習後の分類器に`lr_2`、予測に使うデータに`X_tuning`、予測ターゲットのデータに`y_tuning`を設定して実行してください。

**練習問題10**：`plot_roc_curve`関数を使ってROC曲線を出力してください。`plot_roc_curve`関数は以下のように使うのでした。
```
plot_roc_curve(学習後の分類器、予測に使うデータ、予測ターゲットのデータ)
```
学習後の分類器に`lr_2`、予測に使うデータに`X_tuning`、予測ターゲットのデータに`y_tuning`を設定して実行してください。

# 実践問題

上記の2つのモデルを比べると、`class_weight='balanced'`を使った方が感度とF1スコアに優れるということがわかりました。それではlogistic regressionを使って`class_weight='balanced'`を設定してモデル作成用データ全て(`X_model`と`y_model`)を用いて学習させて、テストデータ(`X_test`と`y_test`)に対する性能を評価してみてください。

まず、モデル作成用データ全てを用いて学習するための分類器を定義します。

In [ ]:
lr_act = LogisticRegression(solver='liblinear', class_weight='balanced')

**実践問題1**：上記で定義した`lr_act`を`X_model`と`y_model`を用いて学習させてください。

**実践問題2**：学習後の`lr_act`を使って`X_test`に対して予測を行い、予測結果を`y_pred`に格納してください。

下のセルを実行し、正解率、適合率、再現率、F1スコアを出力してください。

In [ ]:
print(accuracy_score(y_test, y_pred))    #accuracy: 正解率
print(precision_score(y_test, y_pred))   #precision: 適合率（陽性的中率)
print(recall_score(y_test, y_pred))      #recall: 再現率（感度）
print(f1_score(y_test, y_pred))          #F1 score: 再現率と適合率の調和平均　2*Precision*Recall/(Precision + Recall)

下のセルを実行し、混同行列とROC曲線を出力してください。

In [ ]:
plot_confusion_matrix(lr_act, X_test, y_test, values_format = 'd')
plot_roc_curve(lr_act, X_test, y_test)

# 発展問題

LogisticRegression関数は`class_weight`以外にも他にも様々なオプションが指定できます。  
(参考: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html )  
オプションに`solver='liblinear'`を設定した条件のもとで、`penalty`、`C`、`class_weight`をいくつかの条件で試してみるとします。例えば、`penalty='l1'`、`C=1`、`class_weight='balanced'`のときは以下のように分類器を定義すれば良いことになります。
```
lr_tmp = LogisticRegression(solver='liblinear', penalty='l1', C=1, class_weight='balanced')
```
これらの条件を機械学習用語ではハイパーパラメータと呼びます。ハイパーパラメータは学習で得るパラメータではなく、設計者が指定する必要があるパラメータになります。ここで試してみるハイパーパラメータの条件をlist化して下のセルで定義します。

In [ ]:
penalty_list = ['l1', 'l2']
C_list = [0.01, 0.1, 1, 10, 100]
class_weight_list = [None, 'balanced']

**発展問題1**：上記のリストとfor文を用い、全ての条件で分類器を作成し、それぞれの分類器で`X_train`と`y_train`を使って学習し、`X_tuning`と`y_tuning`に対するF1スコアを計算し、F1スコアを最大化する`penalty`、`C`、`class_weight`の条件を同定してください。

**発展問題2**：上記で得られたF1スコアを最大化する`penalty`、`C`、`class_weight`の条件を使って、学習用データ全て(`X_model`と`y_model`)を使って学習し、`X_test`と`y_test`に対する正解率、適合率、再現率、F1スコア、混同行列、ROC曲線を出力してください。